In [2]:
import pandas as pd
import itertools
import numpy as np
from tqdm import tqdm
import risk

In [4]:
raw = pd.read_csv('C:\Users\dblab\Desktop\kimsinhyun\reIdentification\raw_data\ks_ptdrctor_prcttq_info.csv)
raw

SyntaxError: EOL while scanning string literal (1227974144.py, line 1)